# **IMPORTANT LIBRARIES**

In [1]:
# Warning Libraries :
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Scientific and Data Manipulation Libraries :
import pandas as pd
import numpy as np
from numpy import percentile
import math
import os
from sklearn.model_selection import train_test_split

In [3]:
# Data Visualization Libraries :
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#pip install lasio

In [5]:
#Libraries to convert .las files to .csv and merge

import lasio
from sys import stdout
import glob  ##For merging csv files

In [6]:
#DATA IMPUTATION LIBRARY
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

In [7]:
#Feature Selection Libraries
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [8]:
#SCALING LIBRARIES
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, RobustScaler, MaxAbsScaler

In [9]:
#pip install catboost

In [10]:
#MODEL TRAINING LIBRARIES
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [11]:
#MODEL ACCURACY LIBRARIES
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [12]:
#grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [13]:
path='/media/mr-robot/Local Disk/summer_training/Train'
os.chdir(path)


# **LAS TO CSV**

In [14]:
# Converting all las files in csv by iterating using lasio
for file in os.listdir():
    if file.endswith(".las"):
        file_path = f"{path}/{file}"
        las=lasio.read(file_path)
        size=len(file_path)
        filepath1=file_path[:size-3]
        las.to_csv(filepath1+'csv', units=False)

In [15]:
# Adding Well name to easily identify
for file in os.listdir():
    if file.endswith(".csv"):
        s=pd.read_csv(file)
        size=len(file)
        dict=[]
        filename= file[:size-4]
        t=s.shape[0]
        for i in range(t):
            dict.append(filename)
        s['WELL']=dict
        s.to_csv(filename+'.csv',index=False)

In [16]:
## To avoid furthur merging data and redundancy
if(os.path.isfile('./merged_data.csv') ):
    os.remove("merged_data.csv")
    
if(os.path.isfile('./FACIES_imputed.csv')):
    os.remove("FACIES_imputed.csv")

if(os.path.isfile('./FACIES_TRAIN.csv')):
    os.remove("FACIES_TRAIN.csv")

In [17]:
# Merging all Well Log using Glob
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv('merged_data.csv',index=False)

# **IMPUTATION**

In [18]:
df = pd.read_csv('merged_data.csv')
df

,DEPTH,ACOUSTICIMPEDANCE1,AI,AVG_PIGN,CALI,CALI[DERIVED]1,DT,FACIES,FLD1,GR,...,CALI_1,NPHI_1,ZCOR,RHOB_1,RXO,SPDH,DTDS,M2R1,TH,U
0,1275.0552,12875.0811,12875081.0,NaN,9.7141,9.7141,50.2544,NaN,NaN,50.2128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1275.2076,12854.2256,12854226.0,NaN,9.7848,9.7848,50.3881,NaN,NaN,49.7509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1275.3600,13024.1377,13024138.0,NaN,9.8300,9.8300,49.8852,NaN,NaN,48.2513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1275.5124,13093.3428,13093343.0,NaN,9.8587,9.8587,49.9032,NaN,NaN,46.8212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1275.6648,13169.9307,13169931.0,NaN,9.8756,9.8756,50.0157,NaN,NaN,45.3463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58494,1622.6028,6069.1309,6069130.5,NaN,8.5257,NaN,123.7404,NaN,NaN,NaN,...,NaN,0.4993,NaN,2.4639,NaN,NaN,123.7404,1.5970,NaN,NaN
58495,1622.7552,6067.8120,6067812.0,NaN,8.5282,NaN,123.8728,NaN,NaN,NaN,...,NaN,0.5313,NaN,2.4660,NaN,NaN,123.8728,1.6128,NaN,NaN
58496,1622.9076,6105.7729,6105773.0,NaN,8.5313,NaN,123.3722,NaN,NaN,NaN,...,NaN,0.5448,NaN,2.4714,NaN,NaN,123.3722,1.7043,NaN,NaN
58497,1623.0600,6152.9897,6152977.5,NaN,8.5331,NaN,122.6038,NaN,NaN,NaN,...,NaN,0.5364,NaN,2.4750,NaN,NaN,122.6038,1.8375,NaN,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58499 entries, 0 to 58498
Data columns (total 67 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DEPTH               58499 non-null  float64
 1   ACOUSTICIMPEDANCE1  58499 non-null  float64
 2   AI                  55259 non-null  float64
 3   AVG_PIGN            323 non-null    float64
 4   CALI                54981 non-null  float64
 5   CALI[DERIVED]1      44090 non-null  float64
 6   DT                  58499 non-null  float64
 7   FACIES              52641 non-null  float64
 8   FLD1                3963 non-null   float64
 9   GR                  58379 non-null  float64
 10  LLD                 44942 non-null  float64
 11  LLS                 27394 non-null  float64
 12  DEPTH_1             50885 non-null  float64
 13  NPHI                58172 non-null  float64
 14  ONE-WAYTIME1        15713 non-null  float64
 15  PIGN_MODELLING      51101 non-null  float64
 16  PIMP

In [20]:
df.shape[1]

67

In [21]:
obj = df.isnull().sum() 
for key,value in obj.iteritems():
    print(key,",",value)

DEPTH , 0
ACOUSTICIMPEDANCE1 , 0
AI , 3240
AVG_PIGN , 58176
CALI , 3518
CALI[DERIVED]1 , 14409
DT , 0
FACIES , 5858
FLD1 , 54536
GR , 120
LLD , 13557
LLS , 31105
DEPTH_1 , 7614
NPHI , 327
ONE-WAYTIME1 , 42786
PIGN_MODELLING , 7398
PIMP , 3240
RHOB , 0
RT_MODELLING , 4870
SP , 2507
SUWI_MODELLING , 7400
TDVSS , 62
ZLT , 13937
WELL , 0
DFL , 35041
HDRS , 31548
HMRS , 31548
PERF_INT , 56930
PERMEABILITY , 30350
PIGN , 11550
RT_POWER , 7120
SUWI , 11552
VCL , 11552
WATER_VOL , 14764
LL3 , 46126
BS , 51793
CALI1 , 56110
DEVI , 48216
DT1 , 52369
PHIT , 41967
PIGE , 53254
LLD_1 , 48981
SXWI , 30561
PEF , 39080
AZI1 , 56012
TEMP , 43985
DRES , 55734
DT2 , 55734
DT4P , 52645
GR_EDTC , 55734
M2R2 , 49931
LLS_1 , 58261
MSFL , 55734
PR , 55742
TENS , 55734
VPVS , 55742
BIT , 52946
CALI_1 , 55500
NPHI_1 , 47688
ZCOR , 55501
RHOB_1 , 47600
RXO , 56947
SPDH , 55430
DTDS , 55953
M2R1 , 55953
TH , 55990
U , 55990


In [22]:
#Selecting required feature
df=df[["DT","GR","NPHI","RHOB","FACIES"]]

In [23]:
df

,DT,GR,NPHI,RHOB,FACIES
0,50.2544,50.2128,0.5340,2.1228,NaN
1,50.3881,49.7509,0.5316,2.1250,NaN
2,49.8852,48.2513,0.5126,2.1316,NaN
3,49.9032,46.8212,0.5137,2.1437,NaN
4,50.0157,45.3463,0.5472,2.1611,NaN
...,...,...,...,...,...
58494,123.7404,NaN,0.4993,2.4639,NaN
58495,123.8728,NaN,0.5313,2.4660,NaN
58496,123.3722,NaN,0.5448,2.4714,NaN
58497,122.6038,NaN,0.5364,2.4750,NaN


In [24]:
df.isnull().sum()

DT           0
GR         120
NPHI       327
RHOB         0
FACIES    5858
dtype: int64

In [25]:
#Exporting required features to csv
df.to_csv("FACIES_TRAIN.csv",index=False)

In [26]:
df=pd.read_csv("FACIES_TRAIN.csv")

In [27]:
df.head(20)

,DT,GR,NPHI,RHOB,FACIES
0,50.2544,50.2128,0.5340,2.1228,NaN
1,50.3881,49.7509,0.5316,2.1250,NaN
2,49.8852,48.2513,0.5126,2.1316,NaN
3,49.9032,46.8212,0.5137,2.1437,NaN
4,50.0157,45.3463,0.5472,2.1611,NaN
5,50.6831,44.0819,0.5550,2.1740,NaN
6,51.4311,43.6654,0.5612,2.1707,NaN
7,52.1678,43.3915,0.5566,2.1595,NaN
8,52.2883,44.1249,0.5390,2.1534,NaN
9,51.5991,46.1805,0.5245,2.1551,NaN


In [28]:
df.shape

(58499, 5)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58499 entries, 0 to 58498
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DT      58499 non-null  float64
 1   GR      58379 non-null  float64
 2   NPHI    58172 non-null  float64
 3   RHOB    58499 non-null  float64
 4   FACIES  52641 non-null  float64
dtypes: float64(5)
memory usage: 2.2 MB


In [30]:
sns.heatmap(df.isnull())

<AxesSubplot:>

In [ ]:
null_gr = sns.distplot(df.GR.dropna())

In [ ]:
df.GR.describe()

In [ ]:
null_nphi=sns.distplot(df.NPHI.dropna())

In [ ]:
df.NPHI.describe()

In [ ]:
sns.countplot(x="FACIES",data=df)

In [ ]:
df.FACIES.value_counts(dropna=False)

In [ ]:
def imputing(imputation_strategy,imputing_data):
    df=imputing_data
    if imputation_strategy == "Mean":
        df.GR.fillna(df.GR.mean(),inplace=True)
        print( df.GR.isnull().sum())
        print("Graph (GR) after filling null values with mean")
        sns.displot(df.GR.dropna())
        df.NPHI.fillna(df.NPHI.mean(),inplace=True)
        print("Graph (NPHI) after filling null values with mean")
        print(df.NPHI.isnull().sum())
        sns.displot(df.NPHI.dropna())
        #dropping FACIES rows with null
        df.dropna(axis=0,inplace=True)
        print(df.isnull().sum())
        df['FACIES'] = df.FACIES.astype(np.int64)
        df.info()
        df.FACIES.describe()
        return df

    elif imputation_strategy == "bffill":
        df = df.ffill(axis = 0)
        df = df.bfill(axis = 0)
        df['FACIES'] = df.FACIES.astype(np.int64)
        print(df.isnull().sum())
        return df

    elif imputation_strategy == "KNNImputer":
        knn= KNNImputer(n_neighbors=3)
        X=df.drop('FACIES',1)
        t=knn.fit_transform(X)
        X=pd.DataFrame(t)
        Y=df['FACIES']
        Y=Y.ffill(axis=0)
        Y=Y.bfill(axis=0)
        X['FACIES']=Y
        df=X
        df['FACIES'] = df.FACIES.astype(np.int64)
        d=['DT','GR','NPHI','RHOB']
        for i in range(4):
            df.columns.values[i]=d[i]
        return df
    
    elif imputation_strategy == "IterativeImputer":
        lr=LinearRegression()    #can use other regressions too. / default is beysian
        imp=IterativeImputer(max_iter=3)
        X=df.drop('FACIES',1)
        t=imp.fit_transform(X)
        X=pd.DataFrame(t)
        Y=df['FACIES']
        Y=Y.ffill(axis=0)
        Y=Y.bfill(axis=0)
        X['FACIES']=Y
        df=X
        df['FACIES'] = df.FACIES.astype(np.int64)
        d=['DT','GR','NPHI','RHOB']
        for i in range(4):
            df.columns.values[i]=d[i]
        return df

    elif imputation_strategy == "KNNimputer_floor" :
        X=df
        knn= KNNImputer(n_neighbors=3)
        t=knn.fit_transform(df)
        df=pd.DataFrame(t)
        d=['DT','GR','NPHI','RHOB','FACIES']
        df['FACIES1'] = X.FACIES
        for i in range(5):
            df.columns.values[i]=d[i]
        df=df.drop('FACIES1',1)
        df['FACIES'] = df.FACIES.astype(np.int64)
        return df
    
    elif imputation_strategy == "IterativeImputer_floor" :
        X=df
        lr=LinearRegression()
        imp= IterativeImputer(max_iter=3)
        t=imp.fit_transform(df)
        df=pd.DataFrame(t)
        d=['DT','GR','NPHI','RHOB','FACIES']
        df['FACIES1'] = X.FACIES
        for i in range(5):
            df.columns.values[i]=d[i]
        df=df.drop('FACIES1',1)
        df['FACIES'] = df.FACIES.astype(np.int64)
        return df

    elif imputation_strategy == "KNNBinning" :
        X=df
        knn= KNNImputer(n_neighbors=3)
        t=knn.fit_transform(df)
        df=pd.DataFrame(t)
        d=['DT','GR','NPHI','RHOB','FACIES']
        df['FACIES1'] = X.FACIES
        for i in range(5):
            df.columns.values[i]=d[i]
        df=df.drop('FACIES1',1)
        #df['FACIES'] = pd.cut(x=df['FACIES'],bins=[0,0.5,1.5,2.5,3.5,4.0], labels=['0','1','2','3','4'])
        return df

    elif imputation_strategy == "dropna":
        df=df.dropna(axis=0)
        return df

In [ ]:
imputation_strategy = ["Mean" , "bffill" , "KNNImputer" , "IterativeImputer" , "KNNimputer_floor" , "IterativeImputer_floor" , "KNNBinning","dropna"]
#select option from 0-7 (6 is experimental)
optionimputation=4
df=imputing(imputation_strategy[optionimputation],df)


In [ ]:
#if option==6:
#    df['FACIES'] = pd.cut(x=df['FACIES'],bins=[0.0,0.5,1.5,2.5,3.5,4.0], labels=['0','1','2','3','4'])
    

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv("FACIES_imputed.csv",index=False)
df=pd.read_csv("FACIES_imputed.csv")

In [ ]:
sns.displot(df.GR.dropna())

In [ ]:
print("WHEN GR WAS NULL")
null_gr.figure

In [ ]:
sns.displot(df.NPHI.dropna()) 

In [ ]:
print("WHEN NPHI WAS NULL")
null_nphi.figure

In [ ]:
sns.countplot(x="FACIES",data=df)

# **DATA CONDITIONING / OUTLIER REMOVAL**

In [ ]:
df.head

## WHOLE DATA OUTLIER VISUALIZATION

In [ ]:
plt.boxplot(df)

## DT VISUALIZATION

In [ ]:
sns.histplot(df.DT)

In [ ]:
plt.boxplot(df["DT"])

## GR VISUALIZATION

In [ ]:
sns.histplot(df.GR)

In [ ]:
plt.boxplot(df.GR)

## NPHI VISUALIZATION

In [ ]:
sns.histplot(df.NPHI)

In [ ]:
plt.boxplot(df.NPHI)

## RHOB VISUALIZATION

In [ ]:
sns.histplot(df.RHOB)

In [ ]:
plt.boxplot(df.RHOB)

In [ ]:
def outliers(dataConditioningStrategy,dataframe, dataconditioningcolumns):
    df=dataframe
    if dataConditioningStrategy == "3_Standard_Deviation":
        for column in dataconditioningcolumns:
            print("column",column )
            upperlimit = df[column].mean() + 3*df[column].std()
            lowerlimit = df[column].mean() - 3*df[column].std()

            print("3 standard deviation outliers -:")
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)])
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)].shape)
            df= df[(df[column] < upperlimit) & (df[column] > lowerlimit) & (df.FACIES >= 0) & (df.FACIES <= 4)]
            print(df)

    elif dataConditioningStrategy == "4_Standard_Deviation":
        for column in dataconditioningcolumns:
            print("column",column )
            upperlimit = df[column].mean() + 4*df[column].std()
            lowerlimit = df[column].mean() - 4*df[column].std()

            print("4 standard deviation outliers -:")
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)])
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)].shape)
            df= df[(df[column] < upperlimit) & (df[column] > lowerlimit) & (df.FACIES >= 0) & (df.FACIES <= 4)]
            print(df)
            
    elif dataConditioningStrategy == "InterquartileRange":
        for column in dataconditioningcolumns:
            print("column",column )
            q25, q75 = percentile(df[column], 25), percentile(df[column], 75)
            iqr = q75 - q25
            print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
            cut_off = iqr * 1.5
            lowerlimit, upperlimit = q25 - cut_off, q75 + cut_off

            print("InterQuartile Range Outliers-:")
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)])
            print(df[(df[column] > upperlimit) | (df[column] < lowerlimit)].shape)
            df= df[(df[column] < upperlimit) & (df[column] > lowerlimit) & (df.FACIES >= 0) & (df.FACIES <= 4)]
            print(df)

            
    return df


In [ ]:
DATAConditioningStrategy = ["3_Standard_Deviation","4_Standard_Deviation","InterquartileRange"]
DATAConditioningColumns = ["DT","GR","NPHI","RHOB"]
optionoutlier = 2
df = outliers(DATAConditioningStrategy[optionoutlier] , df, DATAConditioningColumns)

In [ ]:
df.shape

## WHOLE DATA AFTER REMOVING OUTLIERS

In [ ]:
plt.boxplot(df)

In [ ]:
df.head(5)

## DT AFTER REMOVING OUTLIER

In [ ]:
sns.histplot(df.DT)

In [ ]:
plt.boxplot(df["DT"])

## GR AFTER REMOVING OUTLIER

In [ ]:
sns.histplot(df.GR)

In [ ]:
plt.boxplot(df.GR)

## NPHI AFTER REMOVING OUTLIER

In [ ]:
sns.histplot(df.NPHI)

In [ ]:
plt.boxplot(df.NPHI)

## RHOB AFTER REMOVING OUTLIER

In [ ]:
sns.histplot(df.RHOB)

In [ ]:
plt.boxplot(df.RHOB)

In [ ]:
df

# **FEATURE SELECTION**

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
features = df.shape[1]
features

In [ ]:
df.var()

In [ ]:
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor , annot=True , cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
def FeatureSelection(FeatureSelectionStrategy,dataframe):
    df=dataframe
    
    if(FeatureSelectionStrategy=="Variance_Threshold"):
        var_thres=VarianceThreshold(threshold=0.0)
        var_thres.fit(df)
        df.columns[var_thres.get_support()]
        cols = [column for column in df.columns
                    if column not in df.columns[var_thres.get_support()]]
        print(cols)
        df = df.drop(cols,axis=1)
        return df

    if(FeatureSelectionStrategy=="Absolute_Correlation"):
        threshold = 0.6
        col_corr = set()
        corr_matrix = df.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i,j]) > threshold :
                    colname = corr_matrix.columns[i]
                    print(colname)
                    col_corr.add(colname)
        df = df.drop(col_corr,axis=1)
        return df

    if(FeatureSelectionStrategy=="Correlation"):
        threshold = 0.6
        col_corr = set()
        corr_matrix = df.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if (corr_matrix.iloc[i,j]) > threshold :
                    colname = corr_matrix.columns[i]
                    print(colname)
                    col_corr.add(colname)
        df = df.drop(col_corr,axis=1)
        return df

    if(FeatureSelectionStrategy == "SelectKBest"):
        x = df.drop("FACIES",1)
        y = df["FACIES"]
        mutual_info = mutual_info_classif(x,y)
        print(mutual_info)
        mutual_info=pd.Series(mutual_info)
        mutual_info.sort_values(ascending=False)
        mutual_info.sort_values(ascending=False).plot.bar(figsize=(20,8))
        select_col = SelectKBest(mutual_info_classif,k=1)
        select_col.fit(x,y)
        column1 = df.columns[select_col.get_support()]
        df = df.drop(column1,axis=1)
        return df

    if(FeatureSelectionStrategy == "Mutual_Info_Class"):
        x = df.drop("FACIES",1)
        y = df["FACIES"]
        mutual_info = mutual_info_classif(x,y)
        print(mutual_info)
        mutual_info=pd.Series(mutual_info)
        mutual_info.sort_values(ascending=False)
        mutual_info.sort_values(ascending=False).plot.bar(figsize=(20,8))
        return df

In [ ]:
FeatureSelectionStrategy=["Variance_Threshold","Absolute_Correlation","Correlation","SelectKBest","Mutual_Info_Class"]
optionfeature = 0
df=FeatureSelection(FeatureSelectionStrategy[optionfeature],df)

In [ ]:
print("Deleted feature(s) = " + str(features-df.shape[1]))

In [ ]:
df

# **SCALING DATA**

In [ ]:
def data_scaling( scaling_strategy , scaling_data , scaling_columns ):

    if scaling_strategy =="RobustScaler" :
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])

    elif scaling_strategy =="MinMaxScaler" :
        scaling_data[scaling_columns] = MinMaxScaler().fit_transform(scaling_data[scaling_columns])

    else : # If any other scaling send by mistake still perform Robust Scalar
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])

    return scaling_data

In [ ]:
scaling_strategy = ["RobustScaler","MinMaxScaler"]
optionscaling = 0
df = data_scaling( scaling_strategy[optionscaling] , df , DATAConditioningColumns )

In [ ]:
df

In [ ]:
df.to_csv("Preprocessed_data.csv",index=False)

# **SPLITTING DATA USING TRAIN_TEST_SPLIT**

In [ ]:
df=pd.read_csv('Preprocessed_data.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
x = df.drop("FACIES",1)
y = df["FACIES"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=8) 

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_test

# **MODEL TRAINING**

In [ ]:
estimator=[]

In [ ]:
gnb = GaussianNB()


In [ ]:
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

grid = {'solver':solvers,'penalty':penalty,'C':c_values} 
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
dtclf = DecisionTreeClassifier(max_depth=5)

In [ ]:
cat = CatBoostClassifier()

In [ ]:
xgb= XGBClassifier(learning_rate =0.09, 
n_estimators=494, 
max_depth=5,
subsample = 0.70, 
verbosity = 0,)

In [ ]:
lgbm=LGBMClassifier(importance_type = "gain",
verbosity = -1, 
max_bin = 60,
num_leaves=300,
boosting_type = 'dart',
learning_rate=0.1,
n_estimators=494,
max_depth=5, )

In [ ]:
#neigh = KNeighborsClassifier(n_neighbors=3)


In [ ]:
rdmclf = RandomForestClassifier(n_estimators=494,max_depth=5)

In [ ]:
estimator.append(('gaussian',gnb))
estimator.append(('Gridlogistic',grid_search))
estimator.append(('catboost_classifier',cat))
estimator.append(('decision_tree',dtclf))
estimator.append(('xgbclassifier',xgb))
estimator.append(('LGBMclassifier',lgbm))
#estimator.append(('KNN',neigh))

In [ ]:
vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

In [ ]:
vot_soft.fit(X_train,y_train)


In [ ]:
y_pred = vot_soft.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)*100

In [ ]:
t = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= vot_soft.classes_)
disp.plot()

In [ ]:
#metrics.accuracy_score(y_test, y_pred_gnb)*100

In [ ]:
#confusion_matrix(y_test, y_pred_gnb)

In [ ]:
#t = confusion_matrix(y_test, y_pred_gnb)
#disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= gnb.classes_)

In [ ]:
#disp.plot()

In [ ]:
#metrics.accuracy_score(y_test, y_pred_log)*100

In [ ]:
#t = confusion_matrix(y_test, y_pred_log)
#disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= grid_search.classes_)
#disp.plot()

In [ ]:
#metrics.accuracy_score(y_test, y_pred_cat)*100

In [ ]:
#t = confusion_matrix(y_test, y_pred_cat)
#disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= cat.classes_)
#disp.plot()

In [ ]:
#metrics.accuracy_score(y_test, y_pred_dt)*100

In [ ]:
#t = confusion_matrix(y_test, y_pred_dt)
#disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= dtclf.classes_)
#disp.plot()

# **TESTING DATA**

In [ ]:
path = '/media/mr-robot/Local Disk/summer_training/test'
os.chdir(path)

In [ ]:
# Converting all las files in csv by iterating using lasio
for file in os.listdir():
    if file.endswith(".las"):
        file_path = f"{path}/{file}"
        las=lasio.read(file_path)
        size=len(file_path)
        filepath1=file_path[:size-3]
        las.to_csv(filepath1+'csv', units=False)

In [ ]:
## To avoid furthur merging data and redundancy
if(os.path.isfile('./merged_data.csv') ):
    os.remove("merged_data.csv")
    
if(os.path.isfile('./FACIES_imputed.csv')):
    os.remove("FACIES_imputed.csv")

if(os.path.isfile('./FACIES_TRAIN.csv')):
    os.remove("FACIES_TRAIN.csv")

In [ ]:
# Merging all Well Log using Glob
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv('merged_data.csv',index=False)

In [ ]:
df = pd.read_csv('merged_data.csv')
df

In [ ]:
#Selecting required feature
df=df[["DT","GR","NPHI","RHOB","FACIES"]]

In [ ]:
df

In [ ]:
df=imputing(imputation_strategy[optionimputation],df)
df

In [ ]:
df = outliers(DATAConditioningStrategy[optionoutlier] , df, DATAConditioningColumns)

In [ ]:
df = data_scaling( scaling_strategy[optionscaling] , df , DATAConditioningColumns )

In [ ]:
df.to_csv("testing_preprocessed.csv",index=False)

In [ ]:
df=pd.read_csv('testing_preprocessed.csv')

In [ ]:
df

In [ ]:
X_testing=df[["DT","GR","NPHI","RHOB"]]
y_testing=df[["FACIES"]]

In [ ]:
X_testing.isnull().sum()

In [ ]:
#X_testing=FeatureSelection(FeatureSelectionStrategy[optionfeature],X_testing,y_testing)


In [ ]:
X_testing

In [ ]:
y_testing.describe()

In [ ]:
y_predicted = vot_soft.predict(X_testing)

In [ ]:
y_predicted

In [ ]:
metrics.accuracy_score(y_testing, y_predicted)*100

In [ ]:
confusion_matrix(y_testing, y_predicted)

In [ ]:
t = confusion_matrix(y_testing, y_predicted)
disp = ConfusionMatrixDisplay(confusion_matrix= t, display_labels= vot_soft.classes_)
disp.plot()

In [ ]:
t1=pd.DataFrame(y_testing)

In [ ]:
t1.to_csv('y_given.csv',index=False)

In [ ]:
t2=pd.DataFrame(y_predicted)

In [ ]:
t2.to_csv('y_predicted.csv',index=False)